In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import sklearn as sk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from hmmlearn import hmm , vhmm
import scipy.fftpack as fftpack
import warnings
warnings.filterwarnings('ignore')
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim
import itertools
from ta.trend import sma_indicator
from ta.momentum import rsi
from ta.momentum import stoch
from ta.trend import ema_indicator
from ta.trend import adx
from ta.trend import macd
from ta.volume import on_balance_volume
import random
import itertools
from collections import deque
import copy
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from sklearn.preprocessing import MinMaxScaler


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
random.seed(42)

In [ ]:
from collections import namedtuple, deque
Transition = namedtuple('Transition', 
                        ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayBuffer:
    def __init__(self, capacity:int):
        self.capacity = capacity
        self.memory  = deque(maxlen=capacity)

    def push(self, s, a, r, s_next, done):
        self.memory.append((s, a, r, s_next, done))

    def sample(self, batch_size:int):
        batch = random.sample(self.memory, batch_size)
        s,a,r,sn,d = map(np.stack, zip(*batch))
        return (torch.tensor(s,  dtype=torch.float32, device=device),
                torch.tensor(a,  dtype=torch.int64,   device=device),
                torch.tensor(r,  dtype=torch.float32, device=device),
                torch.tensor(sn, dtype=torch.float32, device=device),
                torch.tensor(d,  dtype=torch.bool,    device=device))

    def __len__(self): return len(self.memory)

In [ ]:
def soft_update(target_net: nn.Module,
                online_net: nn.Module,
                tau: float = 0.01):
   
    for t_param, o_param in zip(target_net.parameters(),  online_net.parameters()):
        t_param.data.mul_(1.0 - tau).add_(tau * o_param.data)

In [ ]:
class LSTMPolicyNetwork(nn.Module):
    
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3):
        super(LSTMPolicyNetwork, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layers (batch_first keeps (batch, seq, feat) ordering)
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.1
        )

        # Output head → action probabilities
        self.action_head = nn.Sequential(
            nn.Linear(hidden_size, output_size),
            nn.Softmax(dim=-1)
        )

    def forward(self, x, hidden=None):
      
        if hidden is None:
            hidden = self._init_hidden(x.size(0), x.device)

        # Pass through LSTM
        lstm_out, hidden = self.lstm(x, hidden)

        # Use the output from the final time step
        action_probs = self.action_head(lstm_out[:, -1, :])

        return action_probs, hidden

    def _init_hidden(self, batch_size, device):
        # (h_0, c_0) both zero-initialised
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
        return (h0, c0)

    def select_action(self, state, hidden=None, test_mode=False):
        # Ensure tensor
        if not isinstance(state, torch.Tensor):
            state = torch.FloatTensor(state)

        state = state.to(device)

        # Add batch dim if absent
        if state.dim() == 2:         # [seq_len, features]
            state = state.unsqueeze(0)  # -> [1, seq_len, features]
        elif state.dim() > 3:
            raise ValueError(f"Input state has {state.dim()} dimensions "
                             f"but should have 2 or 3 (got {state.shape})")

        action_probs, hidden = self(state, hidden)

        if test_mode:
            action = torch.argmax(action_probs, dim=1).item()
            return action, hidden, action_probs

        m = Categorical(action_probs)
        action = m.sample()
        log_prob = m.log_prob(action)
        return action.item(), hidden, log_prob

In [ ]:
class LSTMValueNetwork(nn.Module):
  
    def __init__(self, input_size, hidden_size: int = 64, num_layers: int = 3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers  = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.1
        )

        self.value_head = nn.Linear(hidden_size, 1)

    def forward(self, x: torch.Tensor,
                hidden: tuple[torch.Tensor, torch.Tensor] | None = None):
        if hidden is None:
            h_0 = torch.zeros(
                self.num_layers, x.size(0), self.hidden_size, device=x.device
            )
            c_0 = torch.zeros_like(h_0)
            hidden = (h_0, c_0)

        lstm_out, hidden = self.lstm(x, hidden)        # lstm_out: (B, T, H)
        state_value = self.value_head(lstm_out[:, -1, :]).squeeze(-1)

        return state_value, hidden

# Trader

In [ ]:
class PolicyGradientTrader:

    def __init__(self, features, sequence_length=5, hidden_size=32, num_layers=1,
                 transaction_cost=0.001,gamma=0.99,entropy_coef=0.000,batch_size=128,update_cycle=25,replay_capacity =1000,tau=1,update_nets=1):
        self.device = device
        self.features = features
        self.available_features = features 
        self.sequence_length = sequence_length
        self.feature_scaler = MinMaxScaler()
        self.transaction_cost = transaction_cost
        self.action_map = {0: 0, 1: -1, 2: 1}  # Neutral, Short/long, Long/short      
        self.input_size = None
        self.policy = None
        self.optimizer = None
        self.value_net        = None  
        self.value_optimizer  = None  
        self.value_loss_fn    = nn.MSELoss()
        self.gamma=gamma
        self.entropy_coef = entropy_coef 
        self.policy_lr = 1e-4
        self.value_lr = 1e-3
        self.update_net =update_nets      
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.replay_capacity = replay_capacity
        self.batch_size      = batch_size  
        self.update_cycle    = update_cycle     
        self.buffer   = ReplayBuffer(self.replay_capacity)
        self.target_value_net = None    
        self.tau             = tau  
        self.step_counter    = 0


    def preprocess_data(self, data):

        # Check if all features exist in the DataFrame
        available_features = []
        for feature in self.features:
            if feature in data.columns:
                available_features.append(feature)
            else:
                print(f"Warning: Feature '{feature}' not found in data. Skipping.")

        if not available_features:
            raise ValueError(f"None of the specified features {self.features} found in the data columns: {data.columns}")

        # Extract relevant features and scale them
        X = data[available_features].values

        # Store the available features for later use
        self.available_features = available_features

    
        if not hasattr(self, 'feature_scaler_fitted') or not self.feature_scaler_fitted:
            self.feature_scaler.fit(X)
            self.feature_scaler_fitted = True

        # Scale features
        X_scaled = self.feature_scaler.transform(X)

        return X_scaled

    def calculate_cumulative_return(self, returns):
     
        return np.sum(returns)

    def calculate_sharpe_ratio(self, returns, risk_free_rate=0.0):
  
        if len(returns) < 2:
            return 0

        mean_return = np.mean(returns) * 252  # Annualized
        std_return = np.std(returns) * np.sqrt(252)  # Annualized

        if std_return == 0:
            return 0

        return (mean_return - risk_free_rate) / std_return

    def _step_env(self, data, t, action_idx):
   
        new_position = self.action_map[action_idx]

        price_ret1  = data['Return1'].values[self.sequence_length + t]
        price_ret2  = data['Return2'].values[self.sequence_length + t]
        hedge_ratio = 1 #data['Hedge Ratio'].values[self.sequence_length + t - 1]
        tc          = self.transaction_cost* abs(new_position - self.current_position)

        if   new_position ==  1: r_t =  price_ret1 - hedge_ratio*price_ret2 - tc
        elif new_position == -1: r_t = -price_ret1 + hedge_ratio*price_ret2 - tc
        else:                     r_t = -tc                     # flat

        return r_t, new_position

    def train_online(self, data):
        X = self.preprocess_data(data)
        n_steps = len(X) - self.sequence_length

    
        # Initialize policy network if not done yet
        if self.policy is None:
            # Number of features plus position
            self.input_size = X.shape[1] + 1
            self.policy = LSTMPolicyNetwork(
                input_size=self.input_size,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers
            ).to(self.device)
            self.optimizer = optim.Adam(self.policy.parameters(), lr=self.policy_lr)
            
            self.value_net = LSTMValueNetwork(
                input_size=self.input_size,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers
            ).to(self.device)
            self.value_optimizer = optim.Adam(self.value_net.parameters(),    lr=self.value_lr)
            print(f"Initialized policy network with input size {self.input_size} (includes position feature)")
    
            self.target_value_net = copy.deepcopy(self.value_net).eval()
        positions = [0]          # start flat; length will be len(returns)+1
        returns   = []           # one reward per market step
        hidden = None
        self.current_position = 0
        for t in range(n_steps):

            # build current state s_t
            seq       = X[t : t+self.sequence_length]
            pos_feat  = np.full((self.sequence_length,1), self.current_position)
            s_t_np    = np.hstack((seq, pos_feat))
            s_t       = torch.tensor(s_t_np, dtype=torch.float32, device=self.device)

            # actor chooses action 
            action_idx, hidden, log_p = self.policy.select_action(s_t, hidden)
            hidden  = None
            #hidden.detach()
            a_t     = torch.tensor([action_idx], device=self.device)

            # environment step 
            r_t, next_position = self._step_env(data, t, action_idx)

            positions.append(next_position)   # track position for plotting
            returns .append(r_t)             # track realised reward
            self.current_position = next_position
          

            # build s_{t+1}
            if t < n_steps-1:
                seq_next    = X[t+1 : t+1+self.sequence_length]
                pos_feat_n  = np.full((self.sequence_length,1), self.current_position)
                s_next_np   = np.hstack((seq_next, pos_feat_n))
                s_next      = torch.tensor(s_next_np, dtype=torch.float32, device=self.device)
                done        = False
            else:          # last sample
                s_next, done = torch.zeros_like(s_t), True

            # store transition
            self.buffer.push(s_t_np, action_idx, r_t, s_next.cpu().numpy(), done)

            # every update_cycle steps: gradient step
            if len(self.buffer) >= self.batch_size:
                if (t+1)%self.update_net==0:
                    self._learn_from_buffer()
                if (t+1) % self.update_cycle == 0:
                    #  update target critic
                    soft_update(self.target_value_net, self.value_net, self.tau)
        results = {
            'total_return': self.calculate_cumulative_return(np.array(returns)),
            'sharpe_ratio': self.calculate_sharpe_ratio     (np.array(returns)),
            'positions'   : positions[1:],   # drop the initial neutral pos
            'returns'     : returns,
        }
        return results

    def _learn_from_buffer(self):
        S,A,R,SN,D = self.buffer.sample(self.batch_size)

        #  Critic
        with torch.no_grad():
            v_next, _ = self.target_value_net(SN)
            td_target = R + self.gamma * v_next * (~D)

        v, _      = self.value_net(S)
        td_error  = td_target - v

        critic_loss = td_error.pow(2).mean()            

        self.value_optimizer.zero_grad()
        critic_loss.backward()
        nn.utils.clip_grad_norm_(self.value_net.parameters(), 1.0)
        self.value_optimizer.step()

        #  Actor  
        action_probs, _ = self.policy(S)
        dist       = Categorical(action_probs)
        log_prob_a = dist.log_prob(A)

        actor_loss = -(log_prob_a * td_error.detach()).mean()  
        entropy    = dist.entropy().mean()
        total_loss = actor_loss - self.entropy_coef * entropy   

        self.optimizer.zero_grad()
        total_loss.backward()
        nn.utils.clip_grad_norm_(self.policy.parameters(), 1.0)
        self.optimizer.step()



    def save_model(self, path):
    
        torch.save({
            'policy_state_dict': self.policy.state_dict(),
            'feature_scaler': self.feature_scaler,
            'features': self.features,
            'sequence_length': self.sequence_length,
            'feature_scaler_fitted': self.feature_scaler_fitted,
            'input_size': self.input_size,
            'hidden_size': self.hidden_size,
            'num_layers': self.num_layers
        }, path)

    def load_model(self, path, device=None):
     
        # Set device if specified
        if device is not None:
            self.device = self._get_device(device)
            print(f"Using device: {self.device}")

        checkpoint = torch.load(path, map_location=self.device)

       
        self.features = checkpoint['features']
        self.sequence_length = checkpoint['sequence_length']
        self.feature_scaler = checkpoint['feature_scaler']
        self.feature_scaler_fitted = checkpoint['feature_scaler_fitted']
        self.input_size = checkpoint['input_size']
        self.hidden_size = checkpoint.get('hidden_size', self.hidden_size)
        self.num_layers = checkpoint.get('num_layers', self.num_layers)

  
        self.policy = LSTMPolicyNetwork(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers
        ).to(self.device)

        # Load the state dict
        self.policy.load_state_dict(checkpoint['policy_state_dict'])

        # Initialize optimizer
        self.optimizer = optim.Adam(self.policy.parameters(), lr=self.learning_rate)
   

# Get Data

In [ ]:
data = #pd.read_csv(r"DG DLTR.csv")
# train_split=1
# val_split=0.1
# total_size = len(data)
# test_size = int(total_size * (1 - train_split))
# train_val_size = total_size - test_size
# val_size = int(train_val_size *val_split)
# train_size = train_val_size - val_size
 
# train_data = data.iloc[:train_size]
# val_data = data.iloc[train_size:train_size + val_size]
# test_data = data.iloc[train_val_size:]
data.head()

# Hyperparameter Tuning

In [ ]:
import itertools, random, numpy as np, pandas as pd, torch
from tqdm import tqdm

def tune_pg_trader(
        data: pd.DataFrame,
        param_grid: dict,
        n_runs: int = 5,
        hidden_size: int = 32,
        seed: int = 42,
        device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        verbose: bool = True,
):
    def _make_hashable(x):
       
        return tuple(x) if isinstance(x, list) else x
  
    # build grid 
    param_names = list(param_grid.keys())
    grid = list(itertools.product(*(param_grid[p] for p in param_names)))

    metrics_records = []

    outer_iter = tqdm(grid, desc="Param combos", leave=False) if verbose else grid
    for combo in outer_iter:
        params = dict(zip(param_names, combo))

        inner_iter = range(n_runs)
        if verbose:
            inner_iter = tqdm(inner_iter, desc=f"Runs {params}", leave=False)

        for run_idx in inner_iter:
            # deterministic but unique seeds per run 
            this_seed = seed + run_idx
            torch.manual_seed(this_seed)
            np.random.seed(this_seed)
            random.seed(this_seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(this_seed)

            # uild trader
            trader = PolicyGradientTrader(
                features=params["features"],
                sequence_length=params["sequence_length"],
                gamma=params["gamma"],
                entropy_coef=params["entropy_coef"],
                batch_size=params["batch_size"],
                update_cycle=params["update_cycle"],
                replay_capacity = params['replay_capacity'],
                hidden_size=hidden_size,
                num_layers=params['num_layers'],
            )
            safe_params = {k: _make_hashable(v) for k, v in params.items()}

            res = trader.train_online(data)          # online pass
            run_returns = np.array(res["returns"])
            mean_ret     = np.mean(run_returns)
            sharpe      = trader.calculate_sharpe_ratio(run_returns)
            vol         = np.std(run_returns) * np.sqrt(252)

            metrics_records.append({
                **safe_params,
                "run": run_idx,
                "mean_return": mean_ret,
                "sharpe": sharpe,
                "volatility": vol
            })
            #  cleanup GPU
            del trader, res, run_returns       
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats()

    #  assemble results 
    metrics_df = pd.DataFrame(metrics_records)

    # mean metrics per hyper-parameter combination
    agg_cols = ["mean_return", "sharpe", "volatility"]
    summary_df = (
        metrics_df
        .groupby(param_names, as_index=False)[agg_cols]
        .mean()
        .rename(columns={c: f"mean_{c}" for c in agg_cols})
    )

    # pick best by mean cumulative return
    best_idx = summary_df["mean_mean_return"].idxmax()
    summary_df["is_best"] = False
    summary_df.loc[best_idx, "is_best"] = True

    return metrics_df, summary_df

In [ ]:
param_grid_tune = {
    "sequence_length": [5,10,15,20,30],
    "gamma":          [0.99,0.95],
    "entropy_coef":   [0.0],
    "batch_size":     [64,128,256],
    "update_cycle":   [ 10,15,20,25],
    "replay_capacity":[1000,2000],
    "features": [
        ['Spread','Spread Long'],
        ['Spread','rsi2','macd2','rsi1','macd1','obv1','obv2'],
        ['Spread Long','rsi2','macd2','rsi1','macd1','obv1','obv2'],
        ['Spread'],
        ['Spread Long']

    ],
    'transaction_cost':[0.0005],
    'num_layers':[1,2],
    'update_net':[1]
}



metrics_df, summary_df = tune_pg_trader(
    data=data,            # your validation DataFrame
    param_grid=param_grid_tune,
    n_runs=4
)

print(summary_df.sort_values("mean_mean_return", ascending=False))
print(metrics_df.sort_values("mean_return", ascending=False))

# Plotting and Evaluation

In [ ]:
import matplotlib
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats as scipy_stats
def plot_results(
        data,
        positions_list,
        returns_list,
        *,
        timescale: str = "daily",       
        random_seed: int = 42,
        sequence_length: int = 10,
        transaction_cost: float = 0.0005
    ):


   

    # Annualisation factor
    ts = timescale.lower()
    if ts == "daily":
        periods_per_year = 252
    elif ts == "hourly":
        periods_per_year = 252 * 6      # 6 trading hours per day
    else:
        raise ValueError("`timescale` must be 'daily' or 'hourly'")

    np.random.seed(random_seed)

    # Input validation
    if len(positions_list) != len(returns_list):
        raise ValueError("positions_list and returns_list must have the same length")
    n_runs = len(returns_list)
    if n_runs == 0:
        raise ValueError("No runs provided")

    print(f"Analyzing {n_runs} runs ({timescale}) …")

    # Buy-and-Hold benchmarks
    ret1 = data["Return1"].values[sequence_length:]
    ret2 = data["Return2"].values[sequence_length:]
    bh1_cum = np.exp(ret1.cumsum()) - 1.0
    bh2_cum = np.exp(ret2.cumsum()) - 1.0

    #  Fixed-threshold strategies 
    spread = data["Spread"].values
    pr1 = data["Return1"].values
    pr2 = data["Return2"].values

    def _fixed_threshold(thr: float):
        
        pos = np.where(spread > thr, 1, np.where(spread < -thr, -1, 0))
        pnl = pos[:-1] * (pr1[1:] - pr2[1:])               # hedge_ratio≈1
        switch = np.abs(np.diff(np.insert(pos, 0, 0)))
        tc = transaction_cost * switch[:-1]
        return pnl - tc

    returns_05 = _fixed_threshold(0.5)[: len(returns_list[0])]
    returns_1  = _fixed_threshold(1.0)[: len(returns_list[0])]

    #  Random strategies 
    def _random_strategy(length: int, seed_offset: int = 0):
        np.random.seed(random_seed + seed_offset)
        rand_pos = np.random.choice([-1, 0, 1], size=length)
        cur_pos = 0
        rand_ret = []
        for i, new_pos in enumerate(rand_pos):
           
            if new_pos == 1:
                r = pr1[sequence_length + i] - pr2[sequence_length + i]
            elif new_pos == -1:
                r = -pr1[sequence_length + i] + pr2[sequence_length + i]
            else:
                r = 0
            tc = transaction_cost * abs(new_pos - cur_pos)
            rand_ret.append(r - tc)
            cur_pos = new_pos
        return np.asarray(rand_ret), rand_pos

    n_random_runs = max(10, n_runs)
    random_returns_list, random_positions_list = [], []
    for i in range(n_random_runs):
        rr, rp = _random_strategy(len(returns_list[0]), i)
        random_returns_list.append(rr)
        random_positions_list.append(rp)

    #  per-run statistics 
    def _stats(r: np.ndarray):
    
        if r.size == 0:
            return 0, 0, 0, 0, 0, 0
        step_mean = r.mean()
        step_std  = r.std()
        total     = r.sum()
        ann_mean  = step_mean * periods_per_year
        ann_vol   = step_std  * np.sqrt(periods_per_year)
        sharpe    = ann_mean / ann_vol if ann_vol else 0
        return total, ann_vol, ann_mean, sharpe, step_mean, step_std

    # RL statistics
    rl_stats   = np.array([_stats(r) for r in returns_list])          # (n_runs, 6)
    rl_cum     = [np.exp(r.cumsum()) - 1.0 for r in returns_list]

    # Random statistics
    rand_stats = np.array([_stats(r) for r in random_returns_list])
    rand_cum   = [np.exp(r.cumsum()) - 1.0 for r in random_returns_list]

    # Benchmarks
    bh1_stats    = _stats(ret1)
    bh2_stats    = _stats(ret2)
    fix05_stats  = _stats(returns_05)
    fix1_stats   = _stats(returns_1)

    # Aggregate statistics
    rl_mean,  rl_std  = rl_stats.mean(0),  rl_stats.std(0)
    rand_mean, rand_std = rand_stats.mean(0), rand_stats.std(0)

    stats_df = pd.DataFrame({
        "Strategy": [
            "RL Agent (μ)", "RL Agent (σ)",
            "Random (μ)",   "Random (σ)",
            "Buy&Hold 1",   "Buy&Hold 2",
            "Fixed 0.5",    "Fixed 1.0"],
        "Total Return": [
            f"{rl_mean[0]:.4f}", f"±{rl_std[0]:.4f}",
            f"{rand_mean[0]:.4f}", f"±{rand_std[0]:.4f}",
            f"{bh1_stats[0]:.4f}", f"{bh2_stats[0]:.4f}",
            f"{fix05_stats[0]:.4f}", f"{fix1_stats[0]:.4f}"],
        "Ann Return": [
            f"{rl_mean[2]:.4f}", f"±{rl_std[2]:.4f}",
            f"{rand_mean[2]:.4f}", f"±{rand_std[2]:.4f}",
            f"{bh1_stats[2]:.4f}", f"{bh2_stats[2]:.4f}",
            f"{fix05_stats[2]:.4f}", f"{fix1_stats[2]:.4f}"],
        "Volatility": [
            f"{rl_mean[1]:.4f}", f"±{rl_std[1]:.4f}",
            f"{rand_mean[1]:.4f}", f"±{rand_std[1]:.4f}",
            f"{bh1_stats[1]:.4f}", f"{bh2_stats[1]:.4f}",
            f"{fix05_stats[1]:.4f}", f"{fix1_stats[1]:.4f}"],
        "Sharpe": [
            f"{rl_mean[3]:.4f}", f"±{rl_std[3]:.4f}",
            f"{rand_mean[3]:.4f}", f"±{rand_std[3]:.4f}",
            f"{bh1_stats[3]:.4f}", f"{bh2_stats[3]:.4f}",
            f"{fix05_stats[3]:.4f}", f"{fix1_stats[3]:.4f}"],
        "Step Mean": [
            f"{rl_mean[4]:.6f}", f"±{rl_std[4]:.6f}",
            f"{rand_mean[4]:.6f}", f"±{rand_std[4]:.6f}",
            f"{bh1_stats[4]:.6f}", f"{bh2_stats[4]:.6f}",
            f"{fix05_stats[4]:.6f}", f"{fix1_stats[4]:.6f}"],
        "Step Std": [
            f"{rl_mean[5]:.6f}", f"±{rl_std[5]:.6f}",
            f"{rand_mean[5]:.6f}", f"±{rand_std[5]:.6f}",
            f"{bh1_stats[5]:.6f}", f"{bh2_stats[5]:.6f}",
            f"{fix05_stats[5]:.6f}", f"{fix1_stats[5]:.6f}"]
    })

    #plotting
    matplotlib.rcParams.update({'font.size': 18})

    # (a) Cumulative returns
    plt.figure(figsize=(20, 8))
    for c in rl_cum:
        plt.plot(c * 100, color="blue", alpha=0.25, linewidth=0.8)
    rl_mean_cum = np.mean(rl_cum, 0)
    rl_std_cum  = np.std(rl_cum, 0)
    plt.plot(rl_mean_cum * 100, color="blue", linewidth=2.5,
             label=f"RL Strategy (n={n_runs})")
    plt.fill_between(range(len(rl_mean_cum)),
                     (rl_mean_cum - rl_std_cum) * 100,
                     (rl_mean_cum + rl_std_cum) * 100, color="blue",
                     alpha=0.2)

    rand_mean_cum = np.mean(rand_cum, 0)
    rand_std_cum  = np.std(rand_cum, 0)
    plt.plot(rand_mean_cum * 100, "--", color="gray", linewidth=2,
             label=f"Random (n={n_random_runs})")
    plt.fill_between(range(len(rand_mean_cum)),
                     (rand_mean_cum - rand_std_cum) * 100,
                     (rand_mean_cum + rand_std_cum) * 100, color="gray",
                     alpha=0.15)

    plt.plot(bh1_cum * 100, label="Buy & Hold 1", color="green", lw=1.5)
    plt.plot(bh2_cum * 100, label="Buy & Hold 2", color="orange", lw=1.5)
    plt.plot((np.exp(returns_05.cumsum()) - 1) * 100,
             label="Fixed 0.5", color="red", lw=1.2)
    plt.plot((np.exp(returns_1.cumsum()) - 1) * 100,
             label="Fixed 1.0", color="purple", lw=1.2)
    plt.ylabel("Cumulative Return (%)")
    plt.xlabel('Time Steps')
    plt.title("Cumulative Performance Comparison")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

    # (b) Position distribution
    plt.figure(figsize=(8, 8))
    pos_all = np.concatenate(positions_list)
    counts = np.bincount(pos_all + 1)              # [-1,0,1] → [0,1,2]
    labels = ["Short/Long (-1)", "Flat (0)", "Long/Short (+1)"]
    bars = plt.bar(labels, counts, color=["red", "gray", "green"], alpha=0.7)
    total = counts.sum()
    for b, c in zip(bars, counts):
        plt.text(b.get_x() + b.get_width()/2, b.get_height()*1.01,
                 f"({100*c/total:.1f}%)", ha="center", va="bottom")
    plt.ylabel("Frequency")
    plt.title(f"Position Distribution Across {n_runs} Runs")
    plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

    # (c) Return-distribution comparison
    plt.figure(figsize=(10, 6))
    rl_flat   = np.concatenate(returns_list)
    rand_flat = np.concatenate(random_returns_list)
    plt.hist(rl_flat,  bins=50, density=True, alpha=0.7,
             label="RL",     color="blue")
    plt.hist(rand_flat, bins=50, density=True, alpha=0.7,
             label="Random", color="gray")
    plt.axvline(rl_flat.mean(),   color="blue", ls="--",
                label=f"RL μ={rl_flat.mean():.4e}")
    plt.axvline(rand_flat.mean(), color="gray", ls="--",
                label=f"Random μ={rand_flat.mean():.4e}")
    plt.xlabel("One-period return"); plt.ylabel("Density")
    plt.title("Return Distribution Comparison")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

    # (d) Rolling Sharpe ratio
    window = min(50, len(returns_list[0]) // 4)
    def _roll_sharpe(r, w):
        s   = pd.Series(r, dtype=float)
        rm  = s.rolling(w).mean() * periods_per_year
        rsd = s.rolling(w).std()  * np.sqrt(periods_per_year)
        roll = (rm / rsd).replace([np.inf, -np.inf], 0.0).fillna(0.0)
        return roll.values
    plt.figure(figsize=(20, 8))
    rl_roll = np.array([_roll_sharpe(r, window) for r in returns_list])
    for s in rl_roll:
        plt.plot(s, color="blue", alpha=0.25, lw=0.8)
    plt.plot(rl_roll.mean(0), color="blue", lw=2.5, label="RL mean")

    rand_roll = np.array([_roll_sharpe(r, window)
                          for r in random_returns_list[:n_runs]])
    plt.plot(rand_roll.mean(0), "--", color="gray", lw=2,
             label="Random mean")
    plt.axhline(0, color="black", alpha=0.3)
    plt.ylabel("Rolling Sharpe"); plt.xlabel("Time step")
    plt.title(f"Rolling Sharpe Ratio (window = {window})")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

    # (e) Performance metrics bar-chart
    plt.figure(figsize=(8, 6))
    metrics = ["Total Return", "Sharpe", "Consistency"]
    # crude "consistency": 1 – σ_tot/|μ_tot|
    cons_rl   = 1 - rl_std[0]  / (abs(rl_mean[0])  + 1e-8)
    cons_rand = 1 - rand_std[0]/ (abs(rand_mean[0]) + 1e-8)
    mx_ret = max(rl_mean[0], rand_mean[0], bh1_stats[0], bh2_stats[0])
    mx_sh  = max(rl_mean[3], rand_mean[3], bh1_stats[3], bh2_stats[3])
    rl_norm   = [rl_mean[0]/mx_ret if mx_ret else 0,
                 rl_mean[3]/mx_sh  if mx_sh  else 0,
                 max(0, cons_rl)]
    rand_norm = [rand_mean[0]/mx_ret if mx_ret else 0,
                 rand_mean[3]/mx_sh  if mx_sh  else 0,
                 max(0, cons_rand)]
    x = np.arange(len(metrics)); w = 0.35
    plt.bar(x - w/2, rl_norm,   w, label="RL",     color="blue",  alpha=0.7)
    plt.bar(x + w/2, rand_norm, w, label="Random", color="gray",  alpha=0.7)
    plt.xticks(x, metrics); plt.ylabel("Normalized Score")
    plt.title("Performance Metrics Comparison")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

    #  Kolmogorov-Smirnov test 
    ks_stat, ks_p = scipy_stats.ks_2samp(rl_flat, rand_flat, alternative="two-sided")

    print("\n" + "=" * 80)
    print("COMPREHENSIVE PERFORMANCE ANALYSIS")
    print("=" * 80)
    print(stats_df.to_string(index=False))
    print("\nADDITIONAL STATISTICS:")
    print(f"Timescale:          {timescale}  (periods/year = {periods_per_year})")
    print(f"RL runs:            {n_runs}")
    print(f"Random runs:        {n_random_runs}")
    print(f"Series length:      {len(returns_list[0])}")
    print(f"Transaction cost:   {transaction_cost:.4f}")
    print("\nKolmogorov-Smirnov test (RL vs Random):")
    print(f"KS statistic = {ks_stat:.4f}")
    print(f"P-value      = {ks_p:.4f}")
    print(f"Different distributions? {'Yes' if ks_p < 0.05 else 'No'} (α = 0.05)")

    # Return results dict 
    return {
        "stats_df": stats_df,
        "rl_stats":   {"mean": rl_mean,   "std": rl_std,   "runs": rl_stats},
        "rand_stats": {"mean": rand_mean, "std": rand_std, "runs": rand_stats},
        "benchmarks": {
            "buy_hold_1": bh1_stats, "buy_hold_2": bh2_stats,
            "fixed_0.5":  fix05_stats, "fixed_1.0":  fix1_stats},
        "ks_test": {
            "ks_statistic": ks_stat,
            "p_value": ks_p,
            "is_significant": ks_p < 0.05},
        "n_runs": n_runs,
        "n_random_runs": n_random_runs,
        "periods_per_year": periods_per_year,
        "timescale": timescale
    }



In [ ]:
import numpy as np
import torch
import random
import copy
from tqdm import tqdm
def run_multiple_experiments( data,  n_runs=10, verbose=True,  reset_model=True, save_models=False, model_save_path=None):

    # Generate random seeds
    np.random.seed(42)  # For reproducible seed generation
    seeds_used = np.random.randint(0, 100000, n_runs).tolist()
    
    if verbose:
        print(f"Starting {n_runs} RL strategy runs...")
        print(f"Seeds: {seeds_used}")
        if reset_model:
            print("Model will be reset between runs")
        else:
            print("Model will continue training across runs")
    
    # Storage for results
    positions_list = []
    returns_list = []
    results_list = []
    run_statistics = []
    
   
    # Progress bar setup
    pbar = tqdm(range(n_runs), desc="RL Runs") if verbose else range(n_runs)
    
    for run_idx in pbar:
        seed = seeds_used[run_idx] 
        

        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed_all(seed)
           
        features = ['Spread', 'Spread Long']
        trader = PolicyGradientTrader(
            features=features,
            sequence_length=10,
            hidden_size=32,
            num_layers=1,
            gamma=0.99,
            update_cycle =20,
            tau=1,
            batch_size=64,
            replay_capacity=1000,
            transaction_cost = 0.0005
        )
        results = trader.train_online(data) 
        
        # Store results 
        positions_list.append(np.array(results['positions']))
        returns_list.append(np.array(results['returns']))
        results_list.append(results)
        
        #  statistics 
        returns_array = np.array(results['returns'])
        run_stats = {
            'run_index': run_idx,
            'seed': seed,
            'total_return': results['total_return'],
            'sharpe_ratio': results['sharpe_ratio'],
            'n_trades': len(returns_array),
            'win_rate': np.mean(returns_array > 0) if len(returns_array) > 0 else 0,
            'volatility': np.std(returns_array) * np.sqrt(252) if len(returns_array) > 0 else 0,
            'mean_return': np.mean(returns_array) if len(returns_array) > 0 else 0,
            'final_position': results['positions'][-1] if len(results['positions']) > 0 else 0
        }
        run_statistics.append(run_stats)
        
       
        if verbose and not isinstance(pbar, range):
            pbar.set_postfix({
                'Total_Ret': f"{results['total_return']:.4f}",
                'Sharpe': f"{results['sharpe_ratio']:.4f}"
            })
                

    
    if verbose:
        print(f"\nCompleted {n_runs} runs!")
        print(f"Average Total Return: {np.mean([r['total_return'] for r in run_statistics]):.4f}")
        print(f"Average Sharpe Ratio: {np.mean([r['sharpe_ratio'] for r in run_statistics]):.4f}")
        print(f"Success Rate: {sum(1 for r in run_statistics if 'error' not in r)}/{n_runs}")
    
    return {
        'positions_list': positions_list,
        'returns_list': returns_list,
        'results_list': results_list,
        'seeds_used': seeds_used,
        'run_statistics': run_statistics
    }

In [ ]:
multi_results = run_multiple_experiments(
    data=data, 
    n_runs=10, 
    verbose=True
)


analysis = plot_results(
    data=data,
    positions_list=multi_results['positions_list'],
    returns_list=multi_results['returns_list'],
    timescale='hourly',
    transaction_cost=0.0005
)